[Homework 1]('https://colab.research.google.com/github/Jaguar838/llm-zoomcamp/blob/main/HW/2025/01-intro/hw-01.ipynb')

In this homework, we'll learn more about search and use Elastic Search for practice. 

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

## Q1. Running Elastic 

Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

Download and run Elastic Search docker-image:

In [5]:
!docker pull docker.elastic.co/elasticsearch/elasticsearch:8.17.6

8.17.6: Pulling from elasticsearch/elasticsearch
Digest: sha256:d954ee9958a1c5ff4098820fefdaca32c0cf19c573662ec89ba04bd122c19901
Status: Image is up to date for docker.elastic.co/elasticsearch/elasticsearch:8.17.6
docker.elastic.co/elasticsearch/elasticsearch:8.17.6
docker.elastic.co/elasticsearch/elasticsearch:8.17.6


In [6]:
!docker run -p 9200:9200 -e "discovery.type=single-node" \
  -e "xpack.security.enabled=false" \
  docker.elastic.co/elasticsearch/elasticsearch:8.17.6

docker: Error response from daemon: failed to set up container networking: driver failed programming external connectivity on endpoint cranky_newton (b6484a17f915181b4d415a8afca1af0bcdfa2f41572f49d23b4c68ee84f77321): Bind for 0.0.0.0:9200 failed: port is already allocated

Run 'docker run --help' for more information


In [7]:
!curl localhost:9200

{
  "name" : "3ac3ed492c87",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "XKV5WOh1QvSwpMNKIL0zNg",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


Answer Q1: "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7"

## Getting the data

Now let's get the FAQ data. You can run this snippet:

In [8]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

In [9]:
!pip install elasticsearch==8.18.1

In [10]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 
es_client

<Elasticsearch(['http://localhost:9200'])>

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [12]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/home/oleg_yamgurov/llm-zoomcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:09<00:00, 104.62it/s]


Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

Answer Q2: `index`

## Q3. Searching

Now let's search in our index. 

We will execute a query "How do execute a command on a Kubernetes pod?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 84.50
* 64.50
* 44.50
* 24.50

Look at the `_score` field.

In [14]:
query = 'How do execute a command on a Kubernetes pod?'

In [21]:
def elastic_search(query):
    search_query = {
        "size": 1,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return response['hits']['hits'][0]['_score'], result_docs


elastic_search(query)

(31.973522,
 [{'text': 'Install the astronomer-cosmos package as a dependency. (see Terraform example).\nMake a new folder, dbt/, inside the dags/ folder of your Composer GCP bucket and copy paste your dbt-core project there. (see example)\nEnsure your profiles.yml is configured to authenticate with a service account key. (see BigQuery example)\nCreate a new DAG using the DbtTaskGroup class and a ProfileConfig specifying a profiles_yml_filepath that points to the location of your JSON key file. (see example)\nYour dbt lineage graph should now appear as tasks inside a task group like this:',
   'section': 'Course Management Form for Homeworks',
   'question': 'How to run a dbt-core project as an Airflow Task Group on Google Cloud Composer using a service account JSON key',
   'course': 'data-engineering-zoomcamp'}])

## Q4. Filtering

Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [22]:
query = 'How do copy a file to a Docker container?'

In [26]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return response['hits']['hits'][0]['_score'], result_docs

max_score, res_docs = elastic_search(query)

print(f"Answer Q4: {res_docs[2]['question']}, score: {max_score}")

Answer Q4: How do I copy files from a different folder into docker container’s working directory?, score: 73.38676


## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do copy a file to a Docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 946
* 1446
* 1946
* 2446

In [28]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"    
        
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


prompt = build_prompt(query, res_docs)

print(f"Answer Q5: length of the prompt: {len(prompt)}")

Answer Q5: length of the prompt: 1447


## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 120
* 220
* 320
* 420

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [29]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens=encoding.encode(prompt)
print(f"Answer Q6: num tokens: {len(tokens)}")

Answer Q6: num tokens: 321


## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.

In [33]:
import os
from groq import Groq
from dotenv import load_dotenv

# Завантаження змінних середовища з файлу .env
load_dotenv()
# Убедитесь, что переменная окружения GROQ_API_KEY установлена
api_key_groq = os.environ.get("GROQ_API_KEY")

if api_key_groq is None:
    raise ValueError("API key is not set. Please set the GROQ_API_KEY environment variable.")

client_groq = Groq(api_key=api_key_groq)

In [ ]:
chat_completion = client_groq.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

In [35]:
from openai import OpenAI
import httpx as httpx

proxy = os.environ.get('HTTP_PROXY')
api_key_openai = os.environ.get("OPENAI_API_KEY")

client_openai = OpenAI(http_client=httpx.Client(proxy=proxy), api_key=api_key_openai)
response = client_openai.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"To execute a command on a Kubernetes pod, you can use the `kubectl exec` command. This is useful when you need to interact with a container directly, such as troubleshooting, inspecting files, or performing specific operations inside the pod. Here's how you can do it:\n\n### Basic Command\n\n```sh\nkubectl exec -it <pod-name> -- <command>\n```\n\n- `-it` is used for interactive tasks. The `i` flag stands for interactive, and the `t` flag allocates a TTY.\n- `<pod-name>` is the name of the pod you want to interact with.\n- `<command>` is the command you want to execute inside the pod.\n\n### Example\n\nIf you want to open a shell inside a pod named `my-pod`:\n\n```sh\nkubectl exec -it my-pod -- /bin/bash\n```\n\nIf the container uses `sh` instead of `bash`, you can use:\n\n```sh\nkubectl exec -it my-pod -- /bin/sh\n```\n\n### Specify a Container\n\nIf your pod has multiple containers, you'll need to specify the container name using the `-c` option:\n\n```sh\nkubectl exec -it <pod-name>

In [38]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [43]:
def llm_llama(prompt):
    response = client_groq.chat.completions.create(
        model='llama3-8b-8192',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [42]:
def llm_gpt(prompt):
    response = client_openai.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [ ]:
query = 'how do I run kafka?'

In [44]:

def rag_llama(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm_llama(prompt)
    return answer

In [45]:

def rag_gpt(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm_gpt(prompt)
    return answer

In [ ]:
print(rag_gpt(query))

In [ ]:
print(rag('the course has already started, can I still enroll?'))

 QUESTION: the course has already started, can I still enroll?

ANSWER: Yes, even if you don't register, you're still eligible to submit the homeworks.

TRANSLATION IN RUSSIAN:
Да, даже если вы не зарегистрировались, вы по-прежнему можете отправлять домашние задания.


In [ ]:
query = 'How do execute a command on a Kubernetes pod?'

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },

            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return response['hits']['hits']

elastic_search(query)

[{'_index': 'course-questions',
  '_id': 'jkpqiZcBfjM-EchMzfrW',
  '_score': 44.50556,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'HUpriZcBfjM-EchMC_sQ',
  '_score': 35.433445,
  '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
   'section': '10. Kubernetes and TensorFlow Serving',
   'question': 'Kubernetes-dashboard',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'rkpqiZcBfjM-EchM2_qM',
  '_score': 33.70974,
  '_source': {'text': 'You 

In [ ]:
query = 'How do copy a file to a Docker container?'

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return response['hits']['hits'][0]['_score'], result_docs

max_score, res_docs = elastic_search(query)

In [ ]:
res_docs[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

In [ ]:
query="How do copy a file to a Docker container?"

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"    
        
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
prompt = build_prompt(query, res_docs)

print("Length of the prompt: ", len(prompt))

Length of the prompt:  1447


In [ ]:
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do copy a file to a Docker container?\n\nCONTEXT: \nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory containe

In [ ]:
import tiktoken

In [ ]:
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens=encoding.encode(prompt)
print('Num tokens: ', len(tokens))

Num tokens:  321


In [ ]:
encoding.decode_single_token_bytes(63842)

b"You're"

In [ ]:
output = llm(prompt)
print(output)

Based on the context, to copy a file to a Docker container, you can use the `docker cp` command. The basic syntax is as follows:

`docker cp /path/to/local/file_or_directory container_id:/path/in/container`

Replace `/path/to/local/file_or_directory` with the path to the file you want to copy, and `container_id` with the ID of the container you want to copy the file to.


In [ ]:
# Прайсы за 1000 токенов ($)
MODEL_PRICING = {
    "gpt-4.1": {"prompt": 0.0020, "cached": 0.0005, "output": 0.0080},
    "gpt-4.1-mini": {"prompt": 0.0004, "cached": 0.0001, "output": 0.0016},
    "gpt-4.1-nano": {"prompt": 0.0001, "cached": 0.000025, "output": 0.0004},
    "openai-o3": {"prompt": 0.0020, "cached": 0.0005, "output": 0.0080},
    "openai-o4-mini": {"prompt": 0.0011, "cached": 0.000275, "output": 0.0044}
}

def estimate_cost(prompt: str, output: str, model: str = "gpt-4.1", discount: float = 0.0, cached_input: bool = False):
    if model not in MODEL_PRICING:
        raise ValueError(f"Модель '{model}' не найдена в прайс-листе")

    pricing = MODEL_PRICING[model]
    encoding = tiktoken.encoding_for_model("gpt-4o")

    # Подсчёт токенов
    prompt_tokens = len(encoding.encode(prompt))
    output_tokens = len(encoding.encode(output))
    total_tokens = prompt_tokens + output_tokens

    # Выбор тарифа для prompt
    prompt_rate = pricing["cached"] if cached_input and "cached" in pricing else pricing["prompt"]

    # Расчёт стоимости
    raw_prompt_cost = (prompt_tokens / 1_000) * prompt_rate
    raw_output_cost = (output_tokens / 1_000) * pricing["output"]
    total_raw_cost = raw_prompt_cost + raw_output_cost
    total_discounted = total_raw_cost * (1 - discount)

    return {
        "model": model,
        "cached_input": cached_input,
        "discount": discount,
        "prompt_tokens": prompt_tokens,
        "output_tokens": output_tokens,
        "total_tokens": total_tokens,
        "cost_prompt": round(raw_prompt_cost, 6),
        "cost_output": round(raw_output_cost, 6),
        "total_cost": round(total_discounted, 6)
    }


In [ ]:
res = estimate_cost(
    prompt=prompt,
    output=output,
    model="gpt-4.1-nano",
    cached_input=True,
    discount=0.5
)

for k, v in res.items():
    print(f"{k}: {v}")


model: gpt-4.1-nano
cached_input: True
discount: 0.5
prompt_tokens: 321
output_tokens: 87
total_tokens: 408
cost_prompt: 8e-06
cost_output: 3.5e-05
total_cost: 2.1e-05


In [30]:
!pip freeze > requirements.txt